# Importing lockdown measures information
In this notebook we will connect to the Coronavirus Government Response Tracker API, maintained by Oxford University. We will then import the relevant data and save it in a Pandas dataframe.

### Credits:
Hale, Thomas, Sam Webster, Anna Petherick, Toby Phillips, and Beatriz Kira (2020). Oxford COVID-19 Government Response Tracker, Blavatnik School of Government. Data use policy: Creative Commons Attribution CC BY standard.

In [2]:
# First import necessary libraries
import requests
import json
import os

import pandas as pd

In [3]:
# List of countries that we are interested in
country_str = "AUT, BEL, BGR, HRV, CYP, CZE, \
                DNK, EST, FIN, FRA, DEU, GRC, \
                HUN, IRL, ITA, LVA, LTU, LUX, \
                MLT, NLD, POL, PRT, ROU, SVK, \
                SVN, ESP, SWE, GBR"
country_list = [c.strip() for c in country_str.split(", ")]

In [4]:
# Set up connection to API
base = "https://covidtrackerapi.bsg.ox.ac.uk/api/v2/stringency/date-range/"

# Format for lookup = {YYYY-MM-DD}/{YYYY-MM-DD}
ext = f"2020-01-01/2020-07-24"

# Get data
response = requests.get(base + ext)
response

<Response [200]>

In [6]:
# Convert JSON response
j = response.json()
j["data"]

KeyError: 0

In [5]:
# Get stringency values for each country per day

# Structure of JSON is j["data"]['2020-04-21']["ABW"]["stringency"]

data = []
for day in j["data"].values():
    for country in day.values():
        if country["country_code"] in country_list:
            data.append({"country": country["country_code"],
                         "date": country["date_value"],
                         "stringency": country["stringency"]})
            
# Import into dataframe and convert date type
df = pd.DataFrame(data)
df["date"] = pd.to_datetime(df["date"])

In [6]:
df.head()

,country,date,stringency
0,AUT,2020-01-01,0.0
1,BEL,2020-01-01,0.0
2,CZE,2020-01-01,0.0
3,DEU,2020-01-01,0.0
4,DNK,2020-01-01,0.0


In [7]:
df.to_pickle(os.path.join("..", "stringency.pkl"))